In [1]:
import pandas as pd
import regex as re
from sklearn.utils import resample
import warnings 
warnings.filterwarnings('ignore')
import nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,LancasterStemmer,SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import emoji
import re 

In [2]:
litigious_data = pd.read_csv(r"C:\Users\Vishnu\Downloads\dataset\litigious.csv",encoding='latin-1')
negative_data = pd.read_csv(r"C:\Users\Vishnu\Downloads\dataset\litigious.csv",encoding='latin-1')
positive_data = pd.read_csv(r"C:\Users\Vishnu\Downloads\dataset\positive.csv",encoding='latin-1')
uncertainty_data = pd.read_csv(r"C:\Users\Vishnu\Downloads\dataset\uncertainty.csv",encoding='latin-1')

In [3]:
litigious_data['new_column']='litigious'
negative_data['new_column']='negative'
positive_data['new_column']='positive'
uncertainty_data['new_column']='uncertainty'

concatinating all dataframe

In [6]:
data = pd.concat([litigious_data,negative_data,positive_data,uncertainty_data])

In [7]:
data=data.reset_index()

In [8]:
data

index  Unnamed: 0                                               Text  \
0            0           0  @Charlie_Corley @Kristine1G @amyklobuchar @Sty...   
1            1           2  https://t.co/YJNiO0p1JV Flagstar Bank disclose...   
2            2           4  OOPS. I typed her name incorrectly (todayâs ...   
3            3          14  @SaltBurned [When the first sign of surrender ...   
4            4          27  "It's Your fight to save the UK from the liars...   
...        ...         ...                                                ...   
879778  206935      937827  There is a bag of cheese hidden somewhere in m...   
879779  206936      937833  @Treg2Cole Thatâs a hellz nah for me. ð¤¢. ...   
879780  206937      937834  @Sam_Mallory El Mizouni and Roles in the same ...   
879781  206938      937838  If you donât take risks youâll always end ...   
879782  206939      937839  @michaelharriot If Friends were better he migh...   

       Language   new_column  
0            en    litigious  
1            en    litigious  
2            en    litigious  
3            en    litigious  
4            en    litigious  
...         ...          ...  
879778       en  uncertainty  
879779       en  uncertainty  
879780       en  uncertainty  
879781       en  uncertainty  
879782       en  uncertainty  

[879783 rows x 5 columns]

Remove unwanted Coloums

In [9]:
data.drop(labels=['index','Unnamed: 0'],axis=1,inplace=True)

In [10]:
data['Language'].value_counts()

en                                                                                                                                                                      807226
es                                                                                                                                                                       18450
pt                                                                                                                                                                       17943
fr                                                                                                                                                                        8356
ja                                                                                                                                                                        7829
                                                                                                                             

In [13]:
data = data[data['Language']=='en']

In [14]:
data

Text Language  \
0       @Charlie_Corley @Kristine1G @amyklobuchar @Sty...       en   
1       https://t.co/YJNiO0p1JV Flagstar Bank disclose...       en   
2       OOPS. I typed her name incorrectly (todayâs ...       en   
3       @SaltBurned [When the first sign of surrender ...       en   
4       "It's Your fight to save the UK from the liars...       en   
...                                                   ...      ...   
879778  There is a bag of cheese hidden somewhere in m...       en   
879779  @Treg2Cole Thatâs a hellz nah for me. ð¤¢. ...       en   
879780  @Sam_Mallory El Mizouni and Roles in the same ...       en   
879781  If you donât take risks youâll always end ...       en   
879782  @michaelharriot If Friends were better he migh...       en   

         new_column  
0         litigious  
1         litigious  
2         litigious  
3         litigious  
4         litigious  
...             ...  
879778  uncertainty  
879779  uncertainty  
879780  uncertainty  
879781  uncertainty  
879782  uncertainty  

[807226 rows x 3 columns]

In [15]:
data.isnull().sum()

Text          0
Language      0
new_column    0
dtype: int64

In [16]:
stop_words = stopwords.words("english")
stop_words.remove('not')

In [17]:
%%time
sentances = []
for i in data['Text']:
    word = []
    sent = i.lower()
    sent = emoji.demojize(sent)
    sent = re.sub('â€˜|â€™',"'",sent)
    sent = re.sub('&amp;',"and",sent)
    sent = re.sub('<.*?>','',sent)
    sent = re.sub('https?://\S*|www\S*','',sent)
    sent = re.sub(' @\w+| #\w+',"",sent)
    sent = re.sub("[^a-z0-9 ]","",sent)
    for i in word_tokenize(sent):
        if i in stop_words:pass
        else:word.append(PorterStemmer().stem(i))
    sentances.append(" ".join(word))

CPU times: total: 12min 54s
Wall time: 12min 57s


In [18]:
cleaned_text_data = pd.DataFrame({"cleaned_Text":sentances})

In [19]:
cleaned_text_data

cleaned_Text
0       charliecorley testimoni not evid court law sta...
1       flagstar bank disclos data breach impact 15mil...
2       oop type name incorrectli today brave wit 6 7 ...
3       saltburn first sign surrend come michael smile...
4       fight save uk liar cheat charlatan steve bray ...
...                                                   ...
807221           bag chees hidden somewher hous aint talk
807222                   treg2col that hellz nah mayb fri
807223  sammallori el mizouni role team could realli g...
807224  dont take risk youll alway end work someon kin...
807225  michaelharriot friend better might notic compa...

[807226 rows x 1 columns]

In [20]:
#bag of words
count_obj = CountVectorizer()
trans = count_obj.fit_transform(cleaned_text_data['cleaned_Text'][0:25000])
trans

<25000x42934 sparse matrix of type '<class 'numpy.int64'>'
	with 351485 stored elements in Compressed Sparse Row format>

In [21]:
array_count = trans.toarray()

In [22]:
count_obj.vocabulary_

{'charliecorley': 8158,
 'testimoni': 37685,
 'not': 27235,
 'evid': 13930,
 'court': 9969,
 'law': 22291,
 'state': 36153,
 'feder': 14557,
 'must': 26164,
 'stand': 36081,
 'cross': 10299,
 'examin': 13980,
 'flagstar': 14930,
 'bank': 5082,
 'disclos': 11953,
 'data': 10836,
 'breach': 6607,
 'impact': 18854,
 '15million': 439,
 'individu': 19085,
 'oop': 28075,
 'type': 39633,
 'name': 26325,
 'incorrectli': 19011,
 'today': 38663,
 'brave': 6584,
 'wit': 41862,
 'probabi': 30486,
 'dont': 12357,
 'forget': 15197,
 'request': 32143,
 'saltburn': 33380,
 'first': 14854,
 'sign': 34909,
 'surrend': 36917,
 'come': 9093,
 'michael': 24962,
 'smile': 35335,
 'split': 35883,
 'face': 14241,
 'like': 22938,
 'sharp': 34576,
 'knife': 21740,
 'flesh': 14964,
 'would': 42175,
 'chill': 8337,
 'dean': 11072,
 'core': 9778,
 'ice': 18590,
 'cold': 8999,
 'exist': 14059,
 'natur': 26446,
 'archangel': 4115,
 'enj': 13461,
 'fight': 14732,
 'save': 33615,
 'uk': 39728,
 'liar': 22823,
 'cheat'

In [23]:
pd.DataFrame(array_count,columns=sorted(count_obj.vocabulary_))

00  000  000001  0007  001  0047  004coinmarketcap  006  007  007anoth  \
0       0    0       0     0    0     0                 0    0    0         0   
1       0    0       0     0    0     0                 0    0    0         0   
2       0    0       0     0    0     0                 0    0    0         0   
3       0    0       0     0    0     0                 0    0    0         0   
4       0    0       0     0    0     0                 0    0    0         0   
...    ..  ...     ...   ...  ...   ...               ...  ...  ...       ...   
24995   0    0       0     0    0     0                 0    0    0         0   
24996   0    0       0     0    0     0                 0    0    0         0   
24997   0    0       0     0    0     0                 0    0    0         0   
24998   0    0       0     0    0     0                 0    0    0         0   
24999   0    0       0     0    0     0                 0    0    0         0   

       ...  zumwalt  zuneth  zuwairahnasir  zyagle09  zychaxrma  zygotes3  \
0      ...        0       0              0         0          0         0   
1      ...        0       0              0         0          0         0   
2      ...        0       0              0         0          0         0   
3      ...        0       0              0         0          0         0   
4      ...        0       0              0         0          0         0   
...    ...      ...     ...            ...       ...        ...       ...   
24995  ...        0       0              0         0          0         0   
24996  ...        0       0              0         0          0         0   
24997  ...        0       0              0         0          0         0   
24998  ...        0       0              0         0          0         0   
24999  ...        0       0              0         0          0         0   

       zylstra  zzztyler  zzzz  zzzzz  
0            0         0     0      0  
1            0         0     0      0  
2            0         0     0      0  
3            0         0     0      0  
4            0         0     0      0  
...        ...       ...   ...    ...  
24995        0         0     0      0  
24996        0         0     0      0  
24997        0         0     0      0  
24998        0         0     0      0  
24999        0         0     0      0  

[25000 rows x 42934 columns]

In [24]:
#TF-IDF
tf_obj = TfidfVectorizer()
trans_tf = tf_obj.fit_transform(cleaned_text_data['cleaned_Text'][0:5000])
trans_tf

<5000x14164 sparse matrix of type '<class 'numpy.float64'>'
	with 69322 stored elements in Compressed Sparse Row format>

In [25]:
pd.DataFrame(trans_tf.toarray(),columns=sorted(tf_obj.vocabulary_.keys()))

0047  007   01  02906for  030  033  036  038crypto  042   05  ...  \
0      0.0  0.0  0.0       0.0  0.0  0.0  0.0        0.0  0.0  0.0  ...   
1      0.0  0.0  0.0       0.0  0.0  0.0  0.0        0.0  0.0  0.0  ...   
2      0.0  0.0  0.0       0.0  0.0  0.0  0.0        0.0  0.0  0.0  ...   
3      0.0  0.0  0.0       0.0  0.0  0.0  0.0        0.0  0.0  0.0  ...   
4      0.0  0.0  0.0       0.0  0.0  0.0  0.0        0.0  0.0  0.0  ...   
...    ...  ...  ...       ...  ...  ...  ...        ...  ...  ...  ...   
4995   0.0  0.0  0.0       0.0  0.0  0.0  0.0        0.0  0.0  0.0  ...   
4996   0.0  0.0  0.0       0.0  0.0  0.0  0.0        0.0  0.0  0.0  ...   
4997   0.0  0.0  0.0       0.0  0.0  0.0  0.0        0.0  0.0  0.0  ...   
4998   0.0  0.0  0.0       0.0  0.0  0.0  0.0        0.0  0.0  0.0  ...   
4999   0.0  0.0  0.0       0.0  0.0  0.0  0.0        0.0  0.0  0.0  ...   

      zoomcarindia  ztpetrizzo   zu  zubac  zubair  zuneth  zuwairahnasir  \
0              0.0         0.0  0.0    0.0     0.0     0.0            0.0   
1              0.0         0.0  0.0    0.0     0.0     0.0            0.0   
2              0.0         0.0  0.0    0.0     0.0     0.0            0.0   
3              0.0         0.0  0.0    0.0     0.0     0.0            0.0   
4              0.0         0.0  0.0    0.0     0.0     0.0            0.0   
...            ...         ...  ...    ...     ...     ...            ...   
4995           0.0         0.0  0.0    0.0     0.0     0.0            0.0   
4996           0.0         0.0  0.0    0.0     0.0     0.0            0.0   
4997           0.0         0.0  0.0    0.0     0.0     0.0            0.0   
4998           0.0         0.0  0.0    0.0     0.0     0.0            0.0   
4999           0.0         0.0  0.0    0.0     0.0     0.0            0.0   

      zychaxrma  zygotes3  zylstra  
0           0.0       0.0      0.0  
1           0.0       0.0      0.0  
2           0.0       0.0      0.0  
3           0.0       0.0      0.0  
4           0.0       0.0      0.0  
...         ...       ...      ...  
4995        0.0       0.0      0.0  
4996        0.0       0.0      0.0  
4997        0.0       0.0      0.0  
4998        0.0       0.0      0.0  
4999        0.0       0.0      0.0  

[5000 rows x 14164 columns]